In [1]:
import os

In [2]:
%pwd

'd:\\Deep_Learning_Projects\\Sports_Classification\\notebooks'

In [3]:
os.chdir("../")
%pwd

'd:\\Deep_Learning_Projects\\Sports_Classification'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class BaseModelConfig:
    root_dir : Path
    base_model_path : Path
    updated_base_model_path : Path
    image_size_params : list
    learning_rate_params : float
    include_top_params : bool
    weight_params : str
    classes_params : int

In [6]:
from src.Sports_Classification.Constants import *
from src.Sports_Classification.utils.utils import *

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
            ):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config["artifacts_root"]])


    def get_base_model(self) -> BaseModelConfig:
        config = self.config['base_model']

        create_directories([self.config['base_model']['root_dir']])

        base_model_config = BaseModelConfig(
            root_dir = Path(config['root_dir']),
            base_model_path = Path(config['base_model_path']),
            updated_base_model_path = Path(config['updated_base_model_path']),
            image_size_params = self.params['IMAGE_SIZE'],
            learning_rate_params = self.params['LEARNING_RATE'],
            include_top_params = self.params['INCLUDE_TOP'],
            weight_params = self.params['WEIGHTS'],
            classes_params = self.params['CLASSES'] 
            )

        return base_model_config

In [8]:
import os, zipfile
from src.Sports_Classification.logger import logging
from src.Sports_Classification.exception import CustomException
import urllib.request as request
from src.Sports_Classification.utils.utils import *
import tensorflow as tf

In [9]:
class PrepareBaseModel:
    def __init__(self, config : BaseModelConfig):
        self.config = config


    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape = self.config.image_size_params,
            weights = self.config.weight_params,
            include_top = self.config.include_top_params
        )

        self.save_model(path = self.config.base_model_path, model = self.model)

SyntaxError: incomplete input (3557260670.py, line 6)